## CS 3120 Final Project: Predicting Nikola Jokic stats for future games

For this project I am going to be using data from "Basketball-Refrence.com". The excat data I am using is his 23-24 season stats and all the games up to today 11/21/2024. I exported the two files and put them in an excel sheet that I will upload into this workbook.

For this project I plan to use and test different models like linear regression, GradientBoosting, and neural networks. For missing data I plan to completly remove the rows that are empty since it is games were the player was inactive. I also plan to remove the columns for 3P% and FT%. This is because there are instances were the values are empty because the player didnt attempt a FT or a 3P. But in the columns for FTA and 3PA it will show a 0 since none were attempted. But the percantege for them will be empty and it should not be 0 or any other value.

For measuring the performance of my models I will be using mean absolute error (MAE) and R^2.





In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Jokic_Stats_23-24_Machine_Learning_Project.xlsx to Jokic_Stats_23-24_Machine_Learning_Project (4).xlsx


In [ ]:
# Loading the file into a data frame
df = pd.read_excel('Jokic_Stats_23-24_Machine_Learning_Project (4).xlsx')

# Print the shape of the df and first 5 rows of raw data
print(f"** df has {df.shape[0]} rows and {df.shape[1]} columns **")
df.head()


** df has 95 rows and 31 columns **


,Rk,G,Date,Age,Tm,Home/Away,Opp,Result,PD,GS,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1.0,2023-10-24,28-247,DEN,Home,LAL,W,(+12),1,...,10.0,13.0,11.0,1.0,1.0,2.0,2.0,29.0,29.3,15.0
1,2,2.0,2023-10-27,28-250,DEN,Away,MEM,W,(+4),1,...,9.0,12.0,7.0,1.0,1.0,9.0,1.0,22.0,17.0,6.0
2,3,3.0,2023-10-29,28-252,DEN,Away,OKC,W,(+33),1,...,12.0,14.0,5.0,0.0,0.0,4.0,3.0,28.0,24.9,21.0
3,4,4.0,2023-10-30,28-253,DEN,Home,UTA,W,(+8),1,...,9.0,10.0,11.0,0.0,2.0,1.0,3.0,27.0,29.7,13.0
4,5,5.0,2023-11-01,28-255,DEN,Away,MIN,L,(-21),1,...,8.0,10.0,3.0,0.0,0.0,5.0,0.0,25.0,14.2,-19.0


In [ ]:
# get a list of the column names that are either integers or floats
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_df = df.select_dtypes(include=numerics).copy()
numeric_df.describe()

,Rk,G,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
count,95.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,75.000000,89.000000,89.000000,...,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,48.000000,45.000000,10.449438,18.011236,0.597213,1.179775,3.033708,0.440320,4.685393,5.707865,...,9.494382,12.505618,9.269663,1.404494,0.876404,3.123596,2.404494,26.764045,27.304494,8.786517
std,27.568098,25.836021,3.407916,5.991461,0.151265,1.173203,2.442286,0.330538,3.238952,3.720955,...,3.108360,3.843896,3.648674,1.222136,0.914780,1.863649,1.184359,8.143629,8.390172,13.130007
min,1.000000,1.000000,2.000000,3.000000,0.281000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,9.000000,-23.000000
25%,24.500000,23.000000,8.000000,14.000000,0.500000,0.000000,1.000000,0.200000,2.000000,3.000000,...,7.000000,10.000000,7.000000,0.000000,0.000000,2.000000,2.000000,23.000000,21.800000,2.000000
50%,48.000000,45.000000,11.000000,18.000000,0.571000,1.000000,3.000000,0.333000,4.000000,5.000000,...,9.000000,12.000000,9.000000,1.000000,1.000000,3.000000,2.000000,27.000000,27.600000,9.000000
75%,71.500000,67.000000,13.000000,22.000000,0.684000,2.000000,5.000000,0.633500,6.000000,7.000000,...,12.000000,15.000000,12.000000,2.000000,1.000000,4.000000,3.000000,32.000000,32.800000,16.000000
max,95.000000,89.000000,18.000000,32.000000,1.000000,7.000000,12.000000,1.000000,18.000000,18.000000,...,17.000000,22.000000,18.000000,6.000000,5.000000,9.000000,6.000000,42.000000,46.600000,37.000000


In [ ]:
# Select the 'Home/Away' column
Home_Away = df['Home/Away']

# Apply one-hot encoding
Home_Away_encoded = pd.get_dummies(Home_Away).astype(int)

# Add the one-hot encoded DataFrame to your main DataFrame
numeric_df = pd.concat([numeric_df, Home_Away_encoded], axis=1)

   Rk    G    FG   FGA    FG%   3P  3PA    3P%   FT  FTA  ...   AST  STL  BLK  \
0   1  1.0  12.0  22.0  0.545  3.0  5.0  0.600  2.0  4.0  ...  11.0  1.0  1.0   
1   2  2.0   8.0  14.0  0.571  1.0  4.0  0.250  5.0  6.0  ...   7.0  1.0  1.0   
2   3  3.0  12.0  16.0  0.750  1.0  2.0  0.500  3.0  3.0  ...   5.0  0.0  0.0   
3   4  4.0  12.0  16.0  0.750  1.0  3.0  0.333  2.0  5.0  ...  11.0  0.0  2.0   
4   5  5.0  11.0  23.0  0.478  1.0  6.0  0.167  2.0  2.0  ...   3.0  0.0  0.0   

   TOV   PF   PTS  GmSc   +/-  Away  Home  
0  2.0  2.0  29.0  29.3  15.0     0     1  
1  9.0  1.0  22.0  17.0   6.0     1     0  
2  4.0  3.0  28.0  24.9  21.0     1     0  
3  1.0  3.0  27.0  29.7  13.0     0     1  
4  5.0  0.0  25.0  14.2 -19.0     1     0  

[5 rows x 24 columns]


In [ ]:
print(numeric_df.head())

   Rk    G    FG   FGA    FG%   3P  3PA    3P%   FT  FTA  ...   AST  STL  BLK  \
0   1  1.0  12.0  22.0  0.545  3.0  5.0  0.600  2.0  4.0  ...  11.0  1.0  1.0   
1   2  2.0   8.0  14.0  0.571  1.0  4.0  0.250  5.0  6.0  ...   7.0  1.0  1.0   
2   3  3.0  12.0  16.0  0.750  1.0  2.0  0.500  3.0  3.0  ...   5.0  0.0  0.0   
3   4  4.0  12.0  16.0  0.750  1.0  3.0  0.333  2.0  5.0  ...  11.0  0.0  2.0   
4   5  5.0  11.0  23.0  0.478  1.0  6.0  0.167  2.0  2.0  ...   3.0  0.0  0.0   

   TOV   PF   PTS  GmSc   +/-  Away  Home  
0  2.0  2.0  29.0  29.3  15.0     0     1  
1  9.0  1.0  22.0  17.0   6.0     1     0  
2  4.0  3.0  28.0  24.9  21.0     1     0  
3  1.0  3.0  27.0  29.7  13.0     0     1  
4  5.0  0.0  25.0  14.2 -19.0     1     0  

[5 rows x 24 columns]


In [ ]:
# See which rows in numeric_df have more than 20 missing values meaning player was inactive
missing = numeric_df.isnull().sum(axis=1)
missing = missing[missing > 20]
print(missing)

17    21
48    21
70    21
92    21
93    21
94    21
dtype: int64


In [ ]:
# Remove these rows from the DataFrame
nrows_before = numeric_df.shape[0]
numeric_df = numeric_df.drop(index=missing.index)
nrows_after = numeric_df.shape[0]
print(f"** Removed {nrows_before - nrows_after} rows **")

** Removed 6 rows **


In [ ]:
# See which columns in numeric_df have more than 50 missing values
missing = numeric_df.isnull().sum()
missing = missing[missing > 0]
print(missing)


3P%    14
FT%     4
dtype: int64


In [ ]:
# Remove these columns from the DataFrame
ncols_before = numeric_df.shape[1]
numeric_df = numeric_df.drop(columns=missing.index)
ncols_after = numeric_df.shape[1]
print(f"** Removed {ncols_before - ncols_after} columns **")

** Removed 2 columns **
